In [1]:
! pip install recordlinkage
! pip install fsspec
! pip install gcsfs

In [1]:
import pandas as pd
import gcsfs
import recordlinkage as rl

In [2]:
from gcsfs.core import GCSFileSystem
gcs = GCSFileSystem('courseradataeng', token='/home/jupyter/.config/gcloud/application_default_credentials.json') 

In [3]:
with gcs.open('soleadify_sample_data/website_dataset_clean.csv') as f:
    website_dataset_clean = pd.read_csv(f)

In [4]:
with gcs.open('soleadify_sample_data/facebook_dataset_clean.csv') as f:
    facebook_dataset_clean = pd.read_csv(f , quotechar='"', escapechar='\\').drop('Unnamed: 0',axis=1)

In [5]:
indexes={'block':[{'facebook':'country_final','website':'country_clean'}
                 ,{'facebook':'region_consistent','website':'region_clean'}
                 ,{'facebook':'city_consistent','website':'city_clean'}
                  ]
        ,'sorted_neighbour':[{'facebook':'domain','website':'root_domain'}
                            ,{'facebook':'name','website':'site_name_clean'}
                             ]
        }

In [ ]:
indexer = rl.Index()
for index_type in indexes.keys():
    if index_type == 'block':
        for index_pair in indexes[index_type]:
            indexer.add(rl.index.Block( index_pair['facebook'], index_pair['website']))
  
#     if index_type == 'sorted_neighbour':
#         for index_pair in indexes[index_type]:
#             indexer.add(rl.index.SortedNeighbourhood( index_pair['facebook'], index_pair['website']))
            
combined_index = indexer.index(facebook_dataset_clean, website_dataset_clean)

In [10]:
facebook_us = facebook_dataset_clean.query('country_final=="United States"')
website_us = website_dataset_clean.query('country_clean=="United States"')

In [14]:
website_us['root_domain'].count()

9524

In [12]:
facebook_us['domain'].count()

6703

In [17]:
indexer = rl.Index()
index_type = 'sorted_neighbour'
for index_pair in indexes[index_type]:
    indexer.add(rl.index.SortedNeighbourhood(index_pair['website'], index_pair['facebook'] ))
combined_index_us = indexer.index(website_us, facebook_us)

In [19]:
combined_index_us_unqiue = combined_index_us.drop_duplicates(keep='first')
combined_index_us_unqiue

MultiIndex([(   20,  3877),
            (   20, 35300),
            (   40,  2429),
            (   40, 67644),
            (   65,  2071),
            (   65, 11752),
            (   65, 22174),
            (   65, 32258),
            (   67, 14001),
            (   72, 24027),
            ...
            (71862, 41779),
            (71865, 15087),
            (71865, 43664),
            (71865, 57319),
            (71869, 24259),
            (71869, 71202),
            (71889, 14065),
            (71905, 10024),
            (71905, 11945),
            (71905, 17983)],
           length=24468)